# Chest X-Ray Multi‑Class Project — Role Notebook

**Dataset:** Kaggle “Lungs Disease Dataset (4 types)” by Omkar Manohar Dalvi  
**Classes:** Normal, Bacterial Pneumonia, Viral Pneumonia, COVID‑19, Tuberculosis

> Use this notebook in **Google Colab**. If you’re running locally, adapt the Drive mount steps accordingly.

## Role — Member 6: Documentation & Deployment

**Responsibilities**  
- Maintain report (methods, results, figures)  
- Build a simple **Streamlit** app for inference + Grad‑CAM  
- (Optional) Export **TensorFlow Lite** model for mobile

## Environment & Paths

- The code below mounts Google Drive (for persistence) and prepares base paths.  
- Set `DATASET_DIR` to where the extracted dataset resides (after Kaggle download).

## Auto‑Report Generator

This section compiles key artifacts (metrics, confusion matrix) into a lightweight Markdown report.

In [ ]:
# === Colab & Paths ===
import os, sys, glob, json, random, shutil, time
from pathlib import Path

# If in Colab, mount Drive (safe to run elsewhere; it will just fail silently)
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    IN_COLAB = True
except Exception as e:
    print("Not running on Colab or Drive not available:", e)
    IN_COLAB = False

# Project root inside Drive (you can change this)
PROJECT_ROOT = Path('/content/drive/MyDrive/Chest_XRay_Project')
PROJECT_ROOT.mkdir(parents=True, exist_ok=True)

# Where the dataset will live (after download & unzip). Adjust as needed.
DATASET_DIR = PROJECT_ROOT / 'lungs_dataset'
OUTPUTS_DIR = PROJECT_ROOT / 'outputs'
MODELS_DIR = PROJECT_ROOT / 'models'
REPORTS_DIR = PROJECT_ROOT / 'reports'

for p in [OUTPUTS_DIR, MODELS_DIR, REPORTS_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATASET_DIR :", DATASET_DIR)
print("OUTPUTS_DIR :", OUTPUTS_DIR)
print("MODELS_DIR  :", MODELS_DIR)
print("REPORTS_DIR :", REPORTS_DIR)

In [ ]:
import json, numpy as np, matplotlib.pyplot as plt

metrics_path = OUTPUTS_DIR / 'metrics.json'
cm_path = OUTPUTS_DIR / 'cm.npy'
report_md = REPORTS_DIR / 'FINAL_REPORT.md'

if metrics_path.exists() and cm_path.exists():
    with open(metrics_path) as f:
        metrics = json.load(f)
    cm = np.load(cm_path)

    lines = []
    lines.append("# Chest X-Ray Multi‑Class — Final Report\n")
    lines.append("## Metrics\n")
    lines.append(json.dumps(metrics, indent=2))
    lines.append("\n## Confusion Matrix\n")
    lines.append(str(cm))
    report_md.write_text("\n".join(lines), encoding='utf-8')
    print("Wrote:", report_md)
else:
    print("Metrics not found yet. Run Member 4 to generate evaluation artifacts.")

In [ ]:
# === Streamlit app skeleton ===
app_code = f"""
import streamlit as st
import numpy as np
import tensorflow as tf
from PIL import Image

st.title("Chest X-Ray Classifier")
st.write("Classes: Will read from classes.json in project folder.")

# Load model and classes
MODEL_PATH = '{MODELS_DIR.as_posix()}/best_model.keras'
CLASSES_PATH = '{(PROJECT_ROOT / "classes.json").as_posix()}'

@st.cache_resource
def load_artifacts():
    model = tf.keras.models.load_model(MODEL_PATH)
    import json
    with open(CLASSES_PATH) as f:
        classes = json.load(f)
    return model, classes

model, CLASS_NAMES = load_artifacts()

def preprocess(img: Image.Image, size=(224,224)):
    img = img.convert('RGB').resize(size)
    arr = np.array(img)/255.0
    return np.expand_dims(arr, axis=0)

uploaded = st.file_uploader("Upload a chest X-ray image", type=['png','jpg','jpeg'])
if uploaded is not None:
    img = Image.open(uploaded)
    st.image(img, caption="Uploaded image", use_column_width=True)
    x = preprocess(img)
    preds = model.predict(x)[0]
    pred_idx = int(np.argmax(preds))
    st.write(f"**Prediction:** {{CLASS_NAMES[pred_idx]}} (confidence: {{preds[pred_idx]:.3f}})")
"""

with open(PROJECT_ROOT / 'app.py', 'w', encoding='utf-8') as f:
    f.write(app_code)
print("Wrote Streamlit app:", PROJECT_ROOT / 'app.py')

In [ ]:
# === (Optional) Export to TensorFlow Lite ===
try:
    import tensorflow as tf
    model = tf.keras.models.load_model(MODELS_DIR / 'best_model.keras')
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    tflite_path = MODELS_DIR / 'model.tflite'
    with open(tflite_path, 'wb') as f:
        f.write(tflite_model)
    print("Exported TFLite model:", tflite_path)
except Exception as e:
    print("TFLite export skipped/failed:", e)